In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
from pymongo import MongoClient

In [2]:
class LogReg(nn.Module):
    def __init__(self):
        super(LogReg, self).__init__()
        self.lin1 = nn.Linear(4, 1)
        self.sig1 = nn.Sigmoid()
    
    def forward(self, x):
        x = self.lin1(x)
        y = self.sig1(x)
        return y

In [3]:
class MongoDataset(Dataset):
    def __init__(self, connect_url, transform=None):
        self.connect_url = connect_url
        self.client = MongoClient(connect_url)
        self.db = self.client.bitmex
        self.transform = transform
    
    def __len__(self):
        return self.db.training_data.count_documents({})
        

    def __getitem__(self, index):
        current_data = self.db.training_data.find().skip(index).limit(1)
        result = np.array([])
        answer = np.array([])
       
        result = np.append(result, current_data[0]["funding"])
        result = np.append(result, current_data[0]["oi"])
        result = np.append(result, current_data[0]["ema"])
        result = np.append(result, np.log(current_data[0]["vol"]))
        answer = np.append(answer, current_data[0]["result"])

        # pylint: disable=E1101
        return self.transform(result), torch.from_numpy(answer).float()

In [4]:
class Normalize(object):
    def __init__(self, mean, std, inplace=False):
        self.mean = mean
        self.std = std
        self.inplace = inplace

    def __call__(self, ndarray):
        answer = np.array([])
        for i in range(ndarray.size):
            normal = (ndarray[i] - self.mean[i]) / self.std[i]
            answer = np.append(answer, normal)
        return answer